In [16]:
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from konlpy.tag import Mecab

# from googletrans import Translator
from models.transformer import * 
from scipy.stats import ortho_group, wasserstein_distance

import time
import numpy as np

import os
import sys
import urllib.request
import requests
import datetime
import pickle
import json

In [2]:
with open('./data/ko_noun_dict(vocab_size-1250).pkl', "rb") as f:
    ko_noun_dict = pickle.load(f)
    
with open('./data/en_noun_dict(vocab_size-1250).pkl', "rb") as f:
    en_noun_dict = pickle.load(f)
    
with open('./data/ko_verb_dict(vocab_size-547).pkl', "rb") as f:
    ko_verb_dict = pickle.load(f)
    
with open('./data/en_verb_dict(vocab_size-547).pkl', "rb") as f:
    en_verb_dict = pickle.load(f)
    
with open('./data/ko_v_1.pkl', "rb") as f:
    ko_adjective_dict = pickle.load(f)
    
with open('./data/en_v_1.pkl', "rb") as f:
    en_adjective_dict = pickle.load(f)
    
with open('./data/ko_verb_1.pkl', "rb") as f:
    ko_verb_dict_2 = pickle.load(f)
    
with open('./data/en_verb_1.pkl', "rb") as f:
    en_verb_dict_2 = pickle.load(f)

In [3]:
ko_data = np.array(list(ko_noun_dict.values()) + list(ko_verb_dict.values()) + list(ko_verb_dict_2.values()) + list(ko_adjective_dict.values()))
en_data = np.array(list(en_noun_dict.values()) + list(en_verb_dict.values()) + list(en_verb_dict_2.values()) + list(en_adjective_dict.values())) 
ko_noun_data = np.array(list(ko_noun_dict.values()))
en_noun_data = np.array(list(en_noun_dict.values())) 
ko_verb_data = np.array(list(ko_verb_dict.values()) + list(ko_verb_dict_2.values()))
en_verb_data = np.array(list(en_verb_dict.values()) + list(en_verb_dict_2.values())) 
ko_adjective_data = np.array(list(ko_adjective_dict.values()))
en_adjective_data = np.array(list(en_adjective_dict.values()))

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(300, bias_initializer='zero'))

In [11]:
opt = tf.keras.optimizers.Adam()

In [12]:
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

In [13]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50)

In [15]:
model.fit(en_noun_data, ko_noun_data, epochs=500, batch_size=64, callbacks=[es]) 

Train on 1250 samples
Epoch 1/500
1250/1250 [==============================] - 0s 32us/sample - loss: 0.0337 - accuracy: 0.3344
Epoch 2/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0337 - accuracy: 0.3256
Epoch 3/500
1250/1250 [==============================] - 0s 28us/sample - loss: 0.0336 - accuracy: 0.3248
Epoch 4/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0336 - accuracy: 0.3352
Epoch 5/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0336 - accuracy: 0.3288
Epoch 6/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0336 - accuracy: 0.3248
Epoch 7/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0336 - accuracy: 0.3312
Epoch 8/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0335 - accuracy: 0.3296
Epoch 9/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0335 - accuracy: 0.3304
Epoch 10/500
1250/1250 [

1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3328
Epoch 78/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3336
Epoch 79/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3384
Epoch 80/500
1250/1250 [==============================] - 0s 30us/sample - loss: 0.0332 - accuracy: 0.3192
Epoch 81/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3328
Epoch 82/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3288
Epoch 83/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3320
Epoch 84/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3320
Epoch 85/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3304
Epoch 86/500
1250/1250 [==========================

In [9]:
pred = model.predict([[ko_data[0]]])

In [18]:
wasserstein_error = 0
for i in range(len(ko_data)):
    
    wasserstein_error += wasserstein_distance(ko_data[i], model(en_data)[i])
    
print(wasserstein_error / len(ko_data))

0.0401001717187711


In [11]:
for i in range(len(ko_data)):
    pred = model.predict([[ko_data[i]]])
    print(np.dot(pred, en_data[i]) / (np.linalg.norm(pred) * np.linalg.norm(en_data[i])))

[0.8281303]
[0.7517294]
[0.6308626]
[0.47406748]
[0.8247392]
[0.67795116]
[0.8645049]
[0.75061774]
[0.7242294]
[0.7831298]
[0.7113907]
[0.7134434]
[0.65602016]
[0.7259446]
[0.70486265]
[0.6626479]
[0.7669421]
[0.6865695]
[0.77221113]
[0.8757358]
[0.8686568]
[0.8605287]
[0.8762382]
[0.89162046]
[0.7377523]
[0.8461279]
[0.82959586]
[0.6523057]
[0.82739264]


[0.74096715]
[0.8149421]
[0.61737317]
[0.69218785]
[0.7290865]
[0.84263474]
[0.8515523]
[0.6234872]
[0.6122094]
[0.84948677]
[0.8290384]
[0.8048177]
[0.57293516]
[0.7326664]
[0.7796542]
[0.8503309]
[0.8626817]
[0.7685769]
[0.6776152]
[0.7425628]
[0.8491978]
[0.76575726]
[0.58833903]
[0.7286849]
[0.8479734]
[0.80276]
[0.82963806]
[0.7496975]
[0.68820226]
[0.5307668]


[0.7737948]
[0.86312956]
[0.88021564]
[0.7812443]
[0.8593995]
[0.8145976]
[0.7218002]
[0.6117456]
[0.7986072]
[0.72082925]
[0.85406363]
[0.6585573]
[0.62139285]
[0.8048977]
[0.8548083]
[0.83893895]
[0.78744847]
[0.68960935]
[0.84874886]
[0.8554845]
[0.80276406]
[0.78746426]
[0.7637261]
[0.891606]
[0.73522264]
[0.80834377]
[0.72618127]
[0.8589009]
[0.8193071]
[0.84684837]


[0.73094976]
[0.83202875]
[0.6877048]
[0.8667841]
[0.61230665]
[0.807424]
[0.657693]
[0.88766915]
[0.7313907]
[0.7843921]
[0.7436145]
[0.66578925]
[0.8755791]
[0.8444233]
[0.8646669]
[0.83276194]
[0.7940852]
[0.74796695]
[0.6876237]
[0.77708244]
[0.792332]
[0.796917]
[0.8411925]
[0.8059613]
[0.85041034]
[0.82427734]
[0.68002075]
[0.86624116]
[0.8781527]
[0.8318366]


[0.8145648]
[0.7944661]
[0.67241484]
[0.86441004]
[0.835296]
[0.687954]
[0.8618791]
[0.7051734]
[0.80479205]
[0.69663817]
[0.636412]
[0.58682644]
[0.75278854]
[0.6753425]
[0.84243214]
[0.78384525]
[0.67021203]
[0.82757354]
[0.73983926]
[0.844503]
[0.8330846]
[0.813969]
[0.8813293]
[0.88706094]
[0.74004287]
[0.7870379]
[0.83711535]
[0.8529109]
[0.6544617]
[0.83877283]


...

[0.696698]
[0.74911875]
[0.4554634]
[0.8453816]
[0.64802253]
[0.8182867]
[0.8614481]
[0.5342688]
[0.8784091]
[0.8559183]
[0.8576246]
[0.75662076]
[0.6548095]
[0.61239994]
[0.874582]
[0.7178918]
[0.7305317]
[0.70716155]
[0.8401155]
[0.84175515]
[0.8445962]
[0.7524092]
[0.73399585]
[0.7660973]
[0.7604805]
[0.79534614]
[0.68441856]
[0.7417009]
[0.8217147]
[0.76265806]


[0.6825608]
[0.8476337]
[0.80821955]
[0.77497655]
[0.6505312]
[0.6773483]
[0.7128112]
[0.72228426]
[0.7448307]
[0.66927177]
[0.7762249]
[0.8566805]
[0.79382795]
[0.57946557]
[0.84543777]
[0.71742356]
[0.81826663]
[0.7918962]
[0.6087468]
[0.82630605]
[0.829055]
[0.8104016]
[0.8067231]
[0.8649143]
[0.70417523]
[0.51373565]
[0.4884665]
[0.7437798]
[0.8010282]
[0.76155645]


[0.86814547]
[0.82768583]
[0.7135557]
[0.84925437]
[0.68855274]
[0.6585987]
[0.7252353]
[0.7779784]
[0.86057156]
[0.7082776]
[0.72773516]
[0.7855322]
[0.73542935]
[0.71849555]
[0.8374551]
[0.71010685]
[0.7963128]
[0.73911595]
[0.69762814]
[0.81452256]
[0.7028697]
[0.75730664]
[0.41001397]
[0.74854773]
[0.60036623]
[0.84264016]
[0.85464364]
[0.5126819]
[0.72511065]
[0.77328867]


[0.7449442]
[0.85954773]
[0.49521273]
[0.5895755]
[0.5021943]
[0.5119037]
[0.7123086]
[0.65530866]
[0.7470274]
[0.8362866]
[0.8601656]
[0.7045711]
[0.8133318]
[0.72126085]
[0.6649664]
[0.8284608]
[0.6586544]
[0.7681232]
[0.54546946]
[0.75757587]
[0.786815]
[0.8234081]
[0.8661482]
[0.7305507]
[0.64710206]
[0.6262786]
[0.8419957]
[0.7159593]
[0.81191176]
[0.83364946]


[0.63929945]
[0.8473879]
[0.8278839]
[0.77520835]
[0.58125156]
[0.7418896]
[0.6045596]
[0.88477653]
[0.80089784]
[0.8509772]
[0.7223589]
[0.8343162]
[0.82842743]
[0.5184284]
[0.8641881]
[0.8199537]
[0.6812957]
[0.8463004]
[0.67164683]
[0.85468894]
[0.9101121]


In [12]:
def embedding_load():
    """
    Load pretrained embedding vec
    """
    en_model = KeyedVectors.load_word2vec_format('./fasttext/wiki.en.vec')
    ko_model = KeyedVectors.load_word2vec_format('./fasttext/wiki.ko.vec')
    
    return en_model, ko_model

In [13]:
en_model, ko_model = embedding_load()

In [14]:
ko_model.vectors

array([[ 4.0987e-01,  5.3006e-03, -1.5832e+00, ..., -3.5900e+00,
         6.8225e-01, -3.8677e+00],
       [-3.0115e-02,  1.3632e-01, -1.3244e-01, ...,  2.6481e-01,
        -1.9128e-01, -4.0283e-03],
       [-8.5165e-02,  1.4352e-01, -2.4257e-01, ...,  1.8149e-01,
        -2.8154e-01, -1.3674e-01],
       ...,
       [-1.9826e-01,  4.2886e-01, -9.7378e-01, ..., -2.7204e-03,
         2.1197e-01,  3.5205e-01],
       [-4.0754e-01,  5.1668e-01, -1.2572e+00, ..., -9.8720e-02,
        -6.5944e-02,  2.4814e-02],
       [-1.8221e-01,  2.6668e-01, -7.6826e-01, ..., -2.2723e-01,
         1.3282e-01,  1.4781e-01]], dtype=float32)

In [15]:
pred_model = model.predict(ko_model.vectors)[:1000]

In [16]:
pred_model

array([[-0.42809206, -0.52919465, -0.8880173 , ..., -0.1710289 ,
         1.133655  , -0.9060721 ],
       [-0.07276805,  0.03206873,  0.03692443, ...,  0.0439766 ,
         0.03989618,  0.0848214 ],
       [-0.06963407, -0.03041362,  0.01163759, ...,  0.17894281,
         0.19273435, -0.02949652],
       ...,
       [ 0.06736584,  0.31068388, -0.10249412, ...,  0.20190641,
        -0.0830651 , -0.207261  ],
       [ 0.04913099,  0.17356564, -0.04366767, ...,  0.09086637,
         0.03165123,  0.07171926],
       [-0.01730574,  0.01195363,  0.16951418, ...,  0.07784311,
         0.25662917,  0.08534843]], dtype=float32)

In [17]:
list(ko_model.vocab)[:1000]

['</s>',
 '.',
 ',',
 ')',
 '(',
 '년',
 "'",
 '-',
 '분류',
 '월',
 '일',
 '#',
 '}',
 '있다',
 '/',
 '~',
 '이',
 '《',
 '》',
 '는',
 '수',
 '제',
 '의',
 '넘겨주기',
 '은',
 '·',
 '있는',
 '그',
 '역',
 'kst',
 '대한민국의',
 '\\',
 '에',
 '토론',
 '선수',
 '바깥',
 '고리',
 '%',
 '한',
 '및',
 '를',
 '?',
 '축구',
 '한다',
 'the',
 '대한',
 '영화',
 'a',
 '을',
 '주',
 '가',
 '명',
 '년에',
 '다른',
 '같은',
 '로',
 '되었다',
 'm',
 '등',
 '회',
 'of',
 '이후',
 '중',
 '그는',
 '미국의',
 '함께',
 '때',
 '또한',
 '에서',
 '현재',
 '때문에',
 '같이',
 '대',
 '후',
 '!',
 '사람',
 '위해',
 '"',
 '것을',
 '더',
 '배우',
 '시',
 '일본',
 '대한민국',
 '태어남',
 '→',
 '또는',
 '그리고',
 '두',
 '하는',
 'kbs',
 '와',
 '현',
 '미국',
 '그러나',
 's',
 '된다',
 '가장',
 '동문',
 '그의',
 '세',
 '있으며',
 'b',
 '따라',
 '것이',
 '‘',
 '것으로',
 '〈',
 '〉',
 '의해',
 '보기',
 '이다',
 '파일',
 '으로',
 '일본의',
 '많은',
 '전',
 '개',
 '것은',
 '올림픽',
 '다시',
 '했다',
 '+',
 '출신',
 '있었다',
 '할',
 'f',
 'tv',
 '과',
 '첫',
 '당시',
 'd',
 '–',
 '모든',
 'jpg',
 '경우',
 'mbc',
 'x',
 'c',
 '것이다',
 '살아있는',
 'and',
 '리그',
 '통해',
 '개의',
 'km',
 '일에',
 '것',
 '차'

In [40]:
sample_add_vocab = list(ko_model.vocab)[21000:22000]

In [41]:
sample_add_vec = ko_model.vectors[21000:22000]

In [48]:
en_model.add(entities=sample_add_vocab, weights=model.predict(sample_add_vec), replace=True)

In [54]:
list(en_model.vocab)[-1000:]

['연결',
 '공항',
 '전혀',
 '문서의',
 '그들의',
 '이들은',
 '이와',
 '도로',
 '사용할',
 '하나로',
 '최대',
 '년간',
 '메달리스트',
 '등과',
 '문제',
 '제국의',
 '포함한',
 '작가',
 '있기',
 '원소',
 '전라북도',
 '관리자',
 '지역의',
 '사이의',
 '심판',
 '사회',
 '여기서',
 '글을',
 '상징',
 '방면',
 '알려진',
 '위키백과의',
 '문헌',
 '문서에',
 '관할',
 '지역에',
 '정규',
 '것과',
 '일단',
 '기술',
 '서비스',
 '되지',
 '죽은',
 '이라',
 '만들어',
 '독립',
 '사고',
 '에게',
 '의하여',
 '주변',
 '드',
 '소프트웨어',
 '명을',
 '런던',
 '시대의',
 '컵',
 '부터',
 '밝혔다',
 '사용한다',
 '정보를',
 '내용이',
 '교황',
 '그들은',
 '말했다',
 '못하고',
 '선의',
 '음력',
 '문제를',
 '해외',
 '중에서',
 '미국인',
 '수가',
 '사용자가',
 '까지',
 '윈도우',
 '포지션',
 '학년',
 '기독교',
 '수는',
 '기존',
 '자기',
 '재위',
 '가능하다',
 '월에는',
 '월부터',
 '캐나다',
 '메이저',
 '세는',
 '각종',
 '비롯한',
 '주는',
 '마찬가지로',
 '부산광역시',
 '개월',
 '위해서',
 '불리는',
 '국회의원',
 '과정에서',
 '영업거리',
 '이들',
 '대표적인',
 '하면',
 '관중수',
 '팝',
 '폐지',
 '성적',
 '못한',
 '없다고',
 '군사',
 '문서가',
 '아래',
 '가능한',
 '아닙니다',
 '미상',
 '실제',
 '실제로',
 '파리',
 '부른다',
 '동위',
 '사람의',
 '잉글랜드',
 '해서',
 '스페인의',
 '만드는',
 '처녀',
 '보고회',
 '효과에',
 '변광성',
 '횟수가',
 '국화',
 '차이에',

In [75]:
en_model.most_similar('비슷한', topn=100)

[('여러', 0.6606472134590149),
 ('매우', 0.6606396436691284),
 ('마찬가지로', 0.6563717126846313),
 ('가진', 0.6546704769134521),
 ('이와', 0.6507827043533325),
 ('다양한', 0.6132790446281433),
 ('관련된', 0.6044305562973022),
 ('달리', 0.6011983156204224),
 ('갖고', 0.5956286191940308),
 ('가지고', 0.5951209664344788),
 ('보면', 0.5919872522354126),
 ('있는데', 0.5882307887077332),
 ('사용하는', 0.5800421833992004),
 ('라는', 0.5677859783172607),
 ('비해', 0.5609347820281982),
 ('보통', 0.5589520931243896),
 ('실제로', 0.5577079057693481),
 ('그들의', 0.5538526177406311),
 ('거의', 0.5537676215171814),
 ('자주', 0.5525184869766235),
 ('그런', 0.5506726503372192),
 ('만드는', 0.5468973517417908),
 ('더욱', 0.5451722741127014),
 ('실제', 0.5431292057037354),
 ('이라는', 0.5403242111206055),
 ('주로', 0.5367835760116577),
 ('이들', 0.5360938906669617),
 ('만들어', 0.5349541902542114),
 ('불리는', 0.5312321186065674),
 ('다음과', 0.5307690501213074),
 ('사용한다', 0.5299025774002075),
 ('gingering', 0.5291323661804199),
 ('비롯한', 0.5272425413131714),
 ('자신의', 0.527128

In [34]:
en_model.most_similar('osaka')

[('nagoya', 0.8567407131195068),
 ('오사카', 0.8113328218460083),
 ('tokyo', 0.8077577948570251),
 ('fukuoka', 0.8011815547943115),
 ('osakako', 0.79173743724823),
 ('osakada', 0.7887656092643738),
 ('sendai', 0.7777775526046753),
 ('osakajo', 0.7724137306213379),
 ('kitakyushu', 0.7662774324417114),
 ('kumamoto', 0.7647007703781128)]

In [67]:
ko_dict['소프트웨어']

KeyError: '소프트웨어'

In [66]:
en_model.most_similar('소프트웨어', topn=50)

[('컴퓨터', 0.714653730392456),
 ('software', 0.6224812269210815),
 ('서비스', 0.6168100833892822),
 ('workxpress', 0.6061419248580933),
 ('authorware', 0.6031953692436218),
 ('hardware/software', 0.6030852794647217),
 ('시스템', 0.6024385690689087),
 ('opensourcing', 0.5940052270889282),
 ('appscan', 0.592441201210022),
 ('softwware', 0.5915827751159668),
 ('realware', 0.5915789604187012),
 ('software—a', 0.5896106958389282),
 ('implementational', 0.5858747959136963),
 ('pmware', 0.5857131481170654),
 ('product/system/software', 0.5853818655014038),
 ('윈도우', 0.5851297378540039),
 ('software/hardware', 0.5851049423217773),
 ('softwares', 0.584517240524292),
 ('firmware/software', 0.5836369395256042),
 ('opensourcescripts', 0.5821502208709717),
 ('intellisource', 0.5805191993713379),
 ('thinkvantage', 0.5802663564682007),
 ('system/software', 0.5801824331283569),
 ('computer/software', 0.5801414847373962),
 ('netapplications', 0.5786535143852234),
 ('hardwarelogic', 0.578528642654419),
 ('feedsy

In [28]:
list(ko_model.vocab)[21000:22000]

['陀',
 '처녀',
 '보고회',
 '효과에',
 '변광성',
 '횟수가',
 '국화',
 '차이에',
 '전파를',
 'insert',
 '평평한',
 '입대',
 '모델에',
 '에이브러햄',
 '본격',
 '가맹',
 '안데스',
 '사고에',
 '사라져',
 '농림부',
 '타인에게',
 '구별할',
 '공동대표',
 '물병자리',
 '논문은',
 '수학하였다',
 '환경과',
 '근본',
 '시정',
 '육군보병학교',
 '봉기를',
 'cw',
 '메츠',
 '협정에',
 '총에',
 '음반사',
 '결론',
 'pierre',
 '글씨를',
 '상태다',
 '어쩌면',
 '둥지를',
 '이사벨',
 '처형',
 '운항을',
 '김정은',
 '장군을',
 '시조는',
 '원리는',
 '추측',
 '문서들의',
 '니켈로디언',
 '어원은',
 '무시한',
 '孔',
 'wind',
 '이용하기',
 'ng',
 '일각에서는',
 '여러차례',
 '사령관으로',
 '전후로',
 '노동자들의',
 '파이어폭스',
 '시차',
 '梵',
 '발견되며',
 '넥서스',
 '어스',
 '원안',
 '평론',
 '깊고',
 '相應',
 '데이터는',
 '시마다',
 '問',
 '띄게',
 '걸렸다',
 '툴루즈',
 '듀크',
 '원리에',
 '창고',
 '연결한다',
 '저작물을',
 '김일성의',
 '암석',
 '킥복싱',
 'mou',
 '알려드립니다',
 '미주',
 '계속하였다',
 '평가와',
 '화석이',
 '함께해요',
 '법률사무소',
 '섭씨',
 '카논',
 '태풍이다',
 '광고에',
 'found',
 '丸',
 'dsp',
 '되어있는',
 '보통이다',
 '이상에',
 '계획된',
 '중앙선거관리위원회',
 '추진하는',
 '인물들이',
 '마술사',
 '구매',
 '위헌',
 'commons',
 '색깔은',
 '소프트웨어이다',
 '김정민',
 '올바르게',
 '기초적인',
 '케인',
 '要',
 '왕세자',
 '레즈',
 

In [76]:
model.weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(300, 300) dtype=float32, numpy=
 array([[ 5.08578531e-02,  2.22826656e-02, -2.00803429e-02, ...,
         -5.94994389e-02, -1.11564565e-02,  1.68310618e-03],
        [ 2.34058071e-02, -6.37597442e-02,  3.87250595e-02, ...,
         -3.56624424e-02,  4.31192145e-02,  6.01107627e-02],
        [-1.55840769e-01,  2.41320790e-03,  3.08063291e-02, ...,
         -7.96444155e-03,  4.82201623e-03,  2.39666998e-02],
        ...,
        [ 1.77878551e-02,  1.91012621e-02, -1.35673545e-02, ...,
          6.94120750e-02, -1.95321403e-02,  7.52080977e-02],
        [ 6.69418350e-02, -3.26955765e-02, -7.99826235e-02, ...,
          5.56594767e-02,  1.08130295e-02, -1.03147335e-01],
        [-1.09816650e-02, -2.58487696e-03, -7.42137805e-03, ...,
         -6.82828128e-02,  1.31326437e-04, -1.73793547e-02]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(300,) dtype=float32, numpy=
 array([-2.79639428e-03, -4.11240272e-02,  1.41712725e-02